# Knowledge Graph RAG Workflow
This notebook demonstrates a Knowledge Graph (KG) based Retrieval-Augmented Generation (RAG) workflow using LlamaIndex. The workflow combines document ingestion, knowledge graph creation, retrieval, and response generation to provide detailed answers to queries about medicinal plants and their effects.
Workflow Overview:

1. Document Ingestion
2. Knowledge Graph Creation
3. Query-based Retrieval
4. Reranking
5. Response Generation

This approach leverages the power of knowledge graphs to represent complex relationships between entities, allowing for more nuanced and context-aware retrieval and generation.

In [94]:
import nest_asyncio

nest_asyncio.apply()

In [95]:
# !pip install -U llama-index

In [96]:
# from utils import load_env
# load_env()

os.environ["OPENAI_API_KEY"] = "sk-proj...""

# Workflow components

## local Neo4j docker instance

We use Neo4j as our graph database to store and query the knowledge graph. Follow these steps to set up a local Neo4j instance using Docker:

for mac or linux, use below
```bash
docker run \
    -p 7474:7474 -p 7687:7687 \
    -v $PWD/data:/data -v $PWD/plugins:/plugins \
    --name neo4j-apoc \
    -e NEO4J_apoc_export_file_enabled=true \
    -e NEO4J_apoc_import_file_enabled=true \
    -e NEO4J_apoc_import_file_use__neo4j__config=true \
    -e NEO4J_PLUGINS=\[\"apoc\"\] \
    neo4j:latest
```

for windows on anaconda prompt, run below

```bash
docker run ^
   -p 7474:7474 -p 7687:7687 ^
   -v "%CD%/data:/data" -v "%CD%/plugins:/plugins" ^
   --name neo4j-apoc ^
   -e NEO4J_apoc_export_file_enabled=true ^
   -e NEO4J_apoc_import_file_enabled=true ^
   -e NEO4J_apoc_import_file_use__neo4j__config=true ^
   -e NEO4J_PLUGINS="[\"apoc\"]" ^
   neo4j:latest
```
Go see your instance at http://localhost:7474/browser/. Default login and password is 'neo4j'.

You will be asked to change the password, change it to 'password'.


In [98]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

username = "neo4j"
password = "password"
url = "bolt://localhost:7687"

# connect to the graph store
graph_store = Neo4jPGStore(username=username, password=password, url=url)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


## Workflow events

To handle steps in the workflow, we need to define a few events:

 - An event to pass retrieved nodes to the reranker
- An event to pass reranked nodes to the synthesizer

The other steps will use the built-in StartEvent and StopEvent events.

In [99]:
from llama_index.core.workflow import Event
from llama_index.core.schema import NodeWithScore


class RetrieverEvent(Event):
    """Result of running retrieval"""

    nodes: list[NodeWithScore]


class RerankEvent(Event):
    """Result of running reranking on retrieved nodes"""

    nodes: list[NodeWithScore]

## Workflow Definition

The `GraphRAGWorkflow` class defines the steps of our KG RAG process:
1. `ingest_docs`: Load documents from a specified directory
2. `index_docs`: Create a knowledge graph from the ingested documents
3. `retrieve`: Query the knowledge graph to retrieve relevant information
4. `rerank`: Refine the retrieved results
5. `synthesize`: Generate a final response based on the reranked information

In [100]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.core.postprocessor.llm_rerank import LLMRerank
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)

from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor
from llama_index.core import Settings

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

class GraphRAGWorkflow(Workflow):
    @step(pass_context=True)
    async def ingest_docs(self, ctx: Context, ev: StartEvent) -> StopEvent | None:
        """Ingest the documents"""
        dirpath = ev.get("dirpath")
        if not dirpath:
            return None
        
        docs = SimpleDirectoryReader(dirpath).load_data()

        # store the docs in the global context
        ctx.data["docs"] = docs

        print(f"ingested {len(docs)} docs")
        return StopEvent(result=docs)
        # return StopEvent(result=f"ingested {len(docs)} docs")


    @step(pass_context=True)
    async def index_docs(self, ctx: Context, ev: StartEvent) -> StopEvent | None:
        """Index the documents dynamically in the graph store"""
        max_triplets = ev.get("max_triplets")
        if not max_triplets:
            return None

        allowed_entity_types = ev.get("allowed_entity_types", None)
        allowed_relation_types = ev.get("allowed_relation_types", None)
        allowed_relation_props = ev.get("allowed_relation_props", [])
        allowed_entity_props = ev.get("allowed_entity_props", [])

        llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo")
        embed_model = OpenAIEmbedding(model="text-embedding-3-small")

        Settings.llm = llm
        Settings.chunk_size = 2048
        Settings.chunk_overlap = 20
        
        docs: List[Document] = ctx.data.get("docs", [])
        if docs is None:
            print("No documents to index, ingest some documents first.")
            return None

        kg_extractor = DynamicLLMPathExtractor(
            llm=llm,
            max_triplets_per_chunk=max_triplets,
            num_workers=4,
            allowed_entity_types=allowed_entity_types,
            allowed_relation_types=allowed_relation_types,
            allowed_relation_props=allowed_relation_props,
            allowed_entity_props=allowed_entity_props,
        )

        index = PropertyGraphIndex.from_documents(
            docs,
            llm=llm,
            embed_model=embed_model,
            property_graph_store=graph_store,
            kg_extractors=[kg_extractor],
            show_progress=True,
        )

        ctx.data["index"] = index

        return StopEvent(result=index)


    @step(pass_context=True)
    async def retrieve(self, ctx: Context, ev: StartEvent) -> RetrieverEvent | None:
        "Entry point for RAG, triggered by a StartEvent with `query`."
        query = ev.get("query")
        if not query:
            return None

        print(f"Query the graph database with: {query}")

        # store the query in the global context
        ctx.data["query"] = query

        # get the index from the global context
        index = ctx.data.get("index")
        if index is None:
            print("Index is empty, load some documents before querying!")
            return None

        # retriever = index.as_retriever(similarity_top_k=10)
        kg_retriever = VectorContextRetriever(
            index.property_graph_store,
            embed_model=embed_model,
            similarity_top_k=5,
            path_depth=1,
            include_text=True,
        )
        nodes = kg_retriever.retrieve(query)
        print(f"Retrieved {len(nodes)} nodes.")
        return RetrieverEvent(nodes=nodes)


    @step(pass_context=True)
    async def rerank(self, ctx: Context, ev: RetrieverEvent) -> RerankEvent:
            """Rerank the nodes based on the query"""
            ranker = LLMRerank(
                choice_batch_size=5, top_n=3, llm=OpenAI(model="gpt-4o-mini")
            )
            print(ctx.data.get("query"), flush=True)
            new_nodes = ranker.postprocess_nodes(
                ev.nodes, query_str=ctx.data.get("query")
            )
            print(f"Reranked nodes to {len(new_nodes)}")
            #print the nodes
            for node in new_nodes:
                print(node.node.text)
            return RerankEvent(nodes=new_nodes)


    @step(pass_context=True)
    async def synthesize(self, ctx: Context, ev: RerankEvent) -> StopEvent:
        """Return a streaming response using reranked nodes."""
        llm = OpenAI(model="gpt-4o-mini")
        summarizer = CompactAndRefine(llm=llm, streaming=True, verbose=True)
        query = ctx.data.get("query")

        response = await summarizer.asynthesize(query, nodes=ev.nodes)
        return StopEvent(result=response)

# Run the workflow
## 1. Ingest Documents

In [101]:
# Ingest the documents
w = GraphRAGWorkflow(timeout=600, verbose=True)
docs = await w.run(dirpath="data/kg_rag_workflow/")

Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 86 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 127 0 (offset 0)
Ignoring wrong pointing object 140 0 (offset 0)
Ignoring wrong pointing object 180 0 (offset 0)


Running step index_docs
Step index_docs produced no event
Running step ingest_docs
ingested 46 docs
Step ingest_docs produced event StopEvent
Running step retrieve
Step retrieve produced no event


In [102]:
# Display the first 2 documents
print(f"docs is type {type(docs[0])}")
for doc in docs[:2]:
    print("-" * 50)
    print(doc)

docs is type <class 'llama_index.core.schema.Document'>
--------------------------------------------------
Doc ID: a73c8504-5c6d-41b3-a38f-6660348fc453
Text: 139 Archivio Italiano di Urologia e Andrologia 2019; 91, 3REVIEW
Nutraceutical treatment and prevention  of benign prostatic
hyperplasia and prostate cancer  Arrigo F.G. Cicero1, Olta
Allkanjari2, Gian Maria Busetto3, Tommaso Cai4, Gaetano Larganà5,
Vittorio Magri6, Gianpaolo Perletti7, Francesco Saverio Robustelli
Della Cuna8, Giorgio Ivan Russ...
--------------------------------------------------
Doc ID: 45ec3fb0-9649-41c3-b595-3bde97c29b9f
Text: Archivio Italiano di Urologia e Andrologia 2019; 91, 3A.F.G.
Cicero, O. Allkanjari, G.M. Busetto, et al.
140INTRODUCTIONTONUTRACEUTICALPRESCRIPTION (Arrigo F.G. Cicero) During
the last years, the pharmaceutical innovation in primary care are
dramatically less frequent and will be even more rare in the next
future. In this context, pre- clinical...


## 2. Create Knowledge Graph

We will be using the `DynamicLLMPathExtractor` to extract the knowledge graph from the documents. 

For comparisons on different extractors, see [this notebook](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/property_graph/Dynamic_KG_Extraction.ipynb)

We will guide the DynamicLLMPathExtractor with the following parameters some allowed entity types, relation types, relation properties, and entity properties. There is no guarantee that the DynamicLLMPathExtractor will extract all of these, but it will serve as a good starting point.

In [103]:
allowed_entity_types=["MEDICINAL_PLANT", "COMPOUND", "SYMPTOM", "TREATMENT", "DISEASE", "STUDY_TYPE"]
allowed_relation_types=["TREATS", "CONTAINS", "ALLEVIATES", "STUDIED_IN", "SIDE_EFFECT_OF", "INTERACTS_WITH"]
allowed_relation_props=["efficacy", "dosage"]
allowed_entity_props=["scientific_name", "common_name", "description"]
    
index = await w.run(
    max_triplets=20, 
    allowed_entity_types=allowed_entity_types, 
    allowed_relation_types=allowed_relation_types, 
    allowed_relation_props=allowed_relation_props, 
    allowed_entity_props=allowed_entity_props, 
    )

Running step index_docs


Parsing nodes: 100%|██████████| 3/3 [00:00<00:00, 771.86it/s]


Running step ingest_docs
Step ingest_docs produced no event
Running step retrieve
Step retrieve produced no event


Extracting and inferring knowledge graph from text: 100%|██████████| 3/3 [00:17<00:00,  5.93s/it]
Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.10s/it]
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


Step index_docs produced event StopEvent


Feel free to inspect the knowledge graph in your browser at http://localhost:7474/browser/

## 3. Query the Knowledge Graph

In [110]:
# run a query 
query = "How does Urtica dioica (stinging nettle) interact with benign prostatic hyperplasia? Be specific and detailed"
result = await w.run(query=query)
async for chunk in result.async_response_gen():
    print(chunk, end="", flush=True)

Running step index_docs
Step index_docs produced no event
Running step ingest_docs
Step ingest_docs produced no event
Running step retrieve
Query the graph database with: How does Urtica dioica (stinging nettle) interact with benign prostatic hyperplasia? Be specific and detailed
Retrieved 2 nodes.
Step retrieve produced event RetrieverEvent
Running step rerank
How does Urtica dioica (stinging nettle) interact with benign prostatic hyperplasia? Be specific and detailed
Reranked nodes to 2
Here are some facts extracted from the provided text:

Changping M ({'creation_date': '2024-08-03', 'last_modified_date': '2024-08-02', 'file_size': 1558509, 'file_path': '/Users/michaelkoch/github/posts/data/kg_rag_workflow/bph-review-2019.pdf', 'name': 'Changping M', 'file_name': 'bph-review-2019.pdf', 'page_label': '12', 'triplet_source_id': 'c436f74d-dbbd-4b99-8bad-50a7c705bfce', 'file_type': 'application/pdf'}) -> AUTHOR_OF -> The efficacy and safety of urtica dioica in treating benign prostatic 

In [105]:
# run another
query = "What are the mechanisms of actions of Lycopene?"
result = await w.run(query=query)
async for chunk in result.async_response_gen():
    print(chunk, end="", flush=True)

Running step index_docs
Step index_docs produced no event
Running step ingest_docs
Step ingest_docs produced no event
Running step retrieve
Query the graph database with: What are the mechanisms of actions of Lycopene?
Retrieved 4 nodes.
Step retrieve produced event RetrieverEvent
Running step rerank
What are the mechanisms of actions of Lycopene?
Reranked nodes to 3
Here are some facts extracted from the provided text:

Lycopene ({'file_size': 575459, 'location': 'prostate, seminal fluid', 'file_path': '/Users/michaelkoch/github/posts/data/kg_rag_workflow/bph-review-2021.pdf', 'file_name': 'bph-review-2021.pdf', 'file_type': 'application/pdf', 'creation_date': '2024-08-03', 'last_modified_date': '2024-08-03', 'effect': 'interfered with local prostate androgenic signaling, IGF-1 expression, inflammatory mediators', 'description': 'Carotenoid found in tomatoes', 'name': 'Lycopene', 'action': 'antioxidant, anti-inflammatory', 'page_label': '29', 'triplet_source_id': 'e3b4ab7a-e3cb-4fb8-8

# Knowledge Graph RAG Workflow: Key Pros and Cons

## Pros

1. **Rich Relationships**: Captures complex entity connections, enabling nuanced retrieval and context-aware responses.
2. **Flexible Querying**: Combines vector search with graph traversal for diverse query types and exploration paths.
3. **Explainability**: Graph structure allows tracing of reasoning, enhancing result transparency and interpretability.

## Cons

1. **Complexity**: More intricate setup, maintenance, and expertise required compared to simpler RAG approaches.
2. **Data Challenges**: Time-consuming data preparation and potential for extraction errors in graph creation.
3. **Resource Intensive**: Higher computational costs and ongoing maintenance needs for optimal performance.

Despite these challenges, KG RAG offers powerful capabilities for handling complex, interconnected information in specific domains.